In [13]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.1.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7330261394472943435
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7018292839
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7757163137484396825
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Input, Dropout, Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

path_dog = os.listdir('./dog')
path_cat = os.listdir('./cat')

img_dog = []
img_cat = []

# 画像の読み込み
for i in range(len(path_dog)):
    img = cv2.imread(os.path.join('./dog', path_dog[i]))
    img = cv2.resize(img, (100, 100))
    img_dog.append(img)
    
for i in range(len(path_cat)):
    img = cv2.imread(os.path.join('./cat', path_cat[i]))
    img = cv2.resize(img, (100, 100))
    img_cat.append(img)

# データセットの作成
X = np.array(img_dog + img_cat)
y = np.array([0] * len(img_dog) + [1] * len(img_cat))

# データのシャッフル
rand_index = np.random.permutation(np.arange(len(X)))
X = X[rand_index]
y = y[rand_index]

# データの分割
X_train = X[:int(len(X) * 0.75)]
y_train = y[:int(len(X) * 0.75)]
X_test = X[int(len(X) * 0.75):]
y_test = y[int(len(X) * 0.75):]

# 正解ラベルをone-hot表現にする
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



In [32]:
# vgg16の読み込み
input_tensor = Input(shape = (100, 100, 3))
vgg16 = VGG16(include_top = False, weights= 'imagenet', input_tensor = input_tensor) # ImageNetの重みを使用

# 層の定義
model = Sequential() # Sequential : 積層型モデル
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256, activation = 'relu')) # 全結合層
model.add(Dropout(0.5)) # ドロップアウト率を0.5に設定
model.add(Dense(2, activation='softmax'))


# vggを凍結
vgg16.trainable = False
model.summary()

# モデルのコンパイル: 生成したモデルに学習プロセスを設定
# loss : 損失関数
# optimizer : 最適化アルゴリズム; SGD: 確率的勾配降下法, lr :learning_rate, momentum: 慣性項(Momentum SGD)
# metrics : 評価関数; 辞書・リストを指定

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# モデルの学習
history = model.fit(X_train, y_train, batch_size = 512, epochs = 20, validation_data = (X_test, y_test))

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_11 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               1179904   
_________________________________________________________________
dropout_11 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 514       
Total params: 15,895,106
Trainable params: 1,180,418
Non-trainable params: 14,714,688
_________________________________________________________________
Train on 18750 samples, validate on 6250 samples
Epoch 1/20
18750/18750 [==========================

In [ ]:
# 学習の推移
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, 21)

plt.plot(epochs, acc, 'bo', label = 'Training Acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation Acc')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label = 'Traing Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.legend()
plt.show()

In [ ]:
# モデルの保存
model.save('dog_cat_clf.h5')

In [ ]:
'''
# 保存したモデルの読み込み例
from tensorflow.keras.models import load_model
model = load_model('./dog_cat_clf.h5')
'''